In [1]:
from keras.models import load_model
model = load_model('socoF.h5')

In [2]:
x = 96
y = 103

In [3]:
import numpy as np
import os
import cv2 as cv
from sklearn.model_selection import train_test_split
import tensorflow as tf

real_data = "CrossMatch"
altered_data = "AMSL_SynFP_SGR_v1/crossmatch_andrey_512_seeds1-50000"

data = []
labels = []

# load the real data and their labels
photos_names = os.listdir(real_data)
for name in photos_names:
    data.append(cv.imread(os.path.join(real_data ,name) ,cv.IMREAD_GRAYSCALE))
    labels.append(0)

# load the altered data and their labels
photos_names = os.listdir(altered_data)
for name in photos_names:
    data.append(cv.imread(os.path.join(altered_data, name) ,cv.IMREAD_GRAYSCALE))
    labels.append(1)

# resizeing all the data to 103X96
for i,image in enumerate(data):
    data[i] = cv.resize(image ,(x,y))

In [4]:
data = np.stack(data).reshape(-1 ,y ,x ,1)
labels = np.stack(labels).reshape(-1 ,1)

train_data, test_data, train_labels, test_labels  = train_test_split(data, labels, test_size=0.2, random_state=42)


# converting numpy arrays to tensors
train_data_tensor = tf.convert_to_tensor(train_data, dtype=tf.float16)
test_data_tensor = tf.convert_to_tensor(test_data, dtype=tf.float16)
train_labels_tensor = tf.convert_to_tensor(train_labels, dtype=tf.int16)
test_labels_tensor = tf.convert_to_tensor(test_labels, dtype=tf.int16)

# converting to 3 channels
train_data_3 = tf.image.grayscale_to_rgb(train_data_tensor)
test_data_3 = tf.image.grayscale_to_rgb(test_data_tensor)

In [5]:
y_pred = model.predict(test_data_3)
# calculate the balanced accuracy
from sklearn.metrics import balanced_accuracy_score
print(balanced_accuracy_score(test_labels_tensor, np.rint(y_pred)))

316/316 [==============================] - 16s 32ms/step
0.5074097695635572
